In [ ]:
import sys
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import os 
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))

# Results loader object does everything
from ionpy.analysis import ResultsLoader
from pathlib import Path
root = Path("/storage/vbutoi/scratch/ESE")
rs = ResultsLoader()

# For using code without restarting.
%load_ext autoreload
%autoreload 
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
from ese.experiment.analysis.inference import load_cal_inference_stats

inference_paths = [
    # root / "inference/01_07_24_WMH_UncalibratedEnsemble",
    # root / "inference/01_07_24_WMH_TempScalingEnsemble",
    # root / "inference/01_07_24_WMH_VectorScalingEnsemble",
    # root / "inference/01_07_24_WMH_DirichletScalingEnsemble",
    # root / "inference/01_07_24_WMH_LTSEnsemble"
    # root / "inference/01_08_24_WMH_WholeEnsembleUncalibrated"
    # root / "inference/01_08_24_WMH_WholeEnsembleTemperatureScaling"
    # root / "inference/01_08_24_WMH_WholeEnsembleVectorScaling"
    # root / "inference/01_08_24_WMH_WholeEnsembleDirichletScaling"
   root / "inference/01_08_24_WMH_WholeEnsembleLTS"
]

inference_info_dict = load_cal_inference_stats(
    log_dirs=inference_paths,
    load_image_df=True,
    load_pixel_meters_dict=False
)

In [ ]:
metadata_df = inference_info_dict['metadata_df']

In [ ]:
metadata_df.head()

In [ ]:
image_info_df = inference_info_dict['image_info_df']

In [ ]:
image_info_df.head()

In [ ]:
# g = sns.catplot(
#     image_info_df, 
#     x="calibrator", 
#     y="qual_score", 
#     kind="box",
#     hue="log_set"
# )
# # g.set(xlim=(0, 1), ylim=(0, 1))
# g.tight_layout()
# plt.show()

In [ ]:
def gather_log_set_stats(
        stats_df, 
        metadata_df,
        target_metric,
        id_vars=['seed']
        ):
    unique_log_sets = stats_df['log_set'].unique()
    for log_set in unique_log_sets:
        log_set_df = stats_df[stats_df['log_set'] == log_set]
        qual_metric_df = log_set_df[log_set_df['qual_metric'] == target_metric]
        # Calculate basic statistics of performance.
        log_set_mean = qual_metric_df['qual_score'].mean()
        log_set_std = qual_metric_df['qual_score'].std()
        log_set_min = qual_metric_df['qual_score'].min()
        log_set_max = qual_metric_df['qual_score'].max()
        # Build the id string from the id_vars.
        id_string = ""
        log_set_metadata_df = metadata_df[metadata_df['log_set'] == log_set]
        for id_var in id_vars:
            id_value = log_set_metadata_df[id_var].unique()
            assert len(id_value) == 1, "Found more than one value for id_var."
            id_string += f"{id_var}:{id_value[0]} "
        print(f"{id_string}| {target_metric} : {log_set_mean:.3f} +- {log_set_std:.3f} [{log_set_min:.3f}, {log_set_max:.3f}]")

In [ ]:
gather_log_set_stats(
    image_info_df, 
    metadata_df=metadata_df, 
    target_metric='Dice',
    # id_vars=['experiment.pretrained_seed']
    id_vars=['model.ensemble_pre_softmax', 'model.ensemble_combine_fn']
    )